# Section: Securing Federated Learning

- Lesson 1: Trusted Aggregator
- Lesson 2: Intro to Additive Secret Sharing
- Lesson 3: Intro to Fixed Precision Encoding
- Lesson 4: Secret Sharing + Fixed Precision in PySyft
- Final Project: Federated Learning wtih Encrypted Gradient Aggregation

# Lesson: Federated Learning with a Trusted Aggregator

In the last section, we learned how to train a model on a distributed dataset using Federated Learning. In particular, the last project aggregated gradients directly from one data owner to another. 

However, while in some cases it could be ideal to do this, what would be even better is to be able to choose a neutral third party to perform the aggregation.

As it turns out, we can use the same tools we used previously to accomplish this.

# Project: Federated Learning with a Trusted Aggregator

In [1]:
import torch
from torch import nn, optim
import syft as sy

In [2]:
hook = sy.TorchHook(torch)

In [3]:
bob = sy.VirtualWorker(hook, id='bob')
alice = sy.VirtualWorker(hook, id='alice')
secure_worker = sy.VirtualWorker(hook, id='secure_worker')

In [4]:
bob.add_workers([alice, secure_worker])
alice.add_workers([bob, secure_worker])
secure_worker.add_workers([alice, bob])

<VirtualWorker id:secure_worker #objects:0>

In [5]:
data = torch.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True)
target = torch.tensor([[1.],[1],[0],[0]], requires_grad=True)

In [6]:
bobs_data = data[0:2].send(bob)
bobs_target = target[0:2].send(bob)

alices_data = data[2:].send(alice)
alices_target = target[2:].send(alice)

In [8]:
model = nn.Linear(2,1)

In [9]:
for round in range(10):
    bobs_model = model.copy().send(bob)
    alices_model = model.copy().send(alice)

    bobs_opt = optim.SGD(params=bobs_model.parameters(), lr=0.1)
    alices_opt = optim.SGD(params=alices_model.parameters(), lr=0.1)

    for i in range(10):
        bobs_opt.zero_grad()
        bobs_pred = bobs_model(bobs_data)
        bobs_loss = ((bobs_pred - bobs_target)**2).sum()
        bobs_loss.backward()
        bobs_opt.step()
        bobs_loss = bobs_loss.get()

        alices_opt.zero_grad()
        alices_pred = alices_model(alices_data)
        alices_loss = ((alices_pred - alices_target)**2).sum()
        alices_loss.backward()
        alices_opt.step()
        alices_loss = alices_loss.get()

    bobs_model.move(secure_worker)
    alices_model.move(secure_worker)

    with torch.no_grad():
        model.weight.set_(((bobs_model.weight.data + alices_model.weight.data)/2).get())
        model.bias.set_(((bobs_model.bias.data + alices_model.bias.data)/2).get())
        
    secure_worker.clear_objects()
    
    print(f"bob loss: {bobs_loss}, alice loss: {alices_loss}" )

bob loss: 0.029371779412031174, alice loss: 0.034264206886291504
bob loss: 0.008485851809382439, alice loss: 0.00658522080630064
bob loss: 0.002001560525968671, alice loss: 0.0009773802012205124
bob loss: 0.00048484178842045367, alice loss: 8.595468534622341e-05
bob loss: 0.00013219125685282052, alice loss: 8.20634760145822e-09
bob loss: 4.321539745433256e-05, alice loss: 1.2421309293131344e-05
bob loss: 1.753348442434799e-05, alice loss: 2.1573316189460456e-05
bob loss: 8.746920684643555e-06, alice loss: 2.242866685264744e-05
bob loss: 5.129882993060164e-06, alice loss: 1.956137566594407e-05
bob loss: 3.3524493119330145e-06, alice loss: 1.581371179781854e-05


tensor(0.0387, requires_grad=True)

In [13]:
data = bobs_model.weight.data.clone().get()

In [14]:
data

tensor([[0.0023, 0.9620]])

# Lesson: Intro to Additive Secret Sharing

While being able to have a trusted third party to perform the aggregation is certainly nice, in an ideal setting we wouldn't have to trust anyone at all. This is where Cryptography can provide an interesting alterantive. 

Specifically, we're going to be looking at a simple protocol for Secure Multi-Party Computation called Additive Secret Sharing. This protocol will allow multiple parties (of size 3 or more) to aggregate their gradients without the use of a trusted 3rd party to perform the aggregation. In other words, we can add 3 numbers together from 3 different people without anyone ever learning the inputs of any other actors.

Let's start by considering the number 5, which we'll put into a varible x

In [13]:
x = 5

Let's say we wanted to SHARE the ownership of this number between two people, Alice and Bob. We could split this number into two shares, 2, and 3, and give one to Alice and one to Bob

In [14]:
bob_x_share = 2
alice_x_share = 3

decrypted_x = bob_x_share + alice_x_share
decrypted_x

5

Note that neither Bob nor Alice know the value of x. They only know the value of their own SHARE of x. Thus, the true value of X is hidden (i.e., encrypted). 

The truly amazing thing, however, is that Alice and Bob can still compute using this value! They can perform arithmetic over the hidden value! Let's say Bob and Alice wanted to multiply this value by 2! If each of them multiplied their respective share by 2, then the hidden number between them is also multiplied! Check it out!

In [15]:
bob_x_share = 2 * 2
alice_x_share = 3 * 2

decrypted_x = bob_x_share + alice_x_share
decrypted_x

10

This even works for addition between two shared values!!

In [16]:
# encrypted "5"
bob_x_share = 2
alice_x_share = 3

# encrypted "7"
bob_y_share = 5
alice_y_share = 2

# encrypted 5 + 7
bob_z_share = bob_x_share + bob_y_share
alice_z_share = alice_x_share + alice_y_share

decrypted_z = bob_z_share + alice_z_share
decrypted_z

12

As you can see, we just added two numbers together while they were still encrypted!!!

One small tweak - notice that since all our numbers are positive, it's possible for each share to reveal a little bit of information about the hidden value, namely, it's always greater than the share. Thus, if Bob has a share "3" then he knows that the encrypted value is at least 3.

This would be quite bad, but can be solved through a simple fix. Decryption happens by summing all the shares together MODULUS some constant. I.e.

In [17]:
x = 5

Q = 23740629843760239486723

bob_x_share = 23552870267 # <- a random number
alice_x_share = Q - bob_x_share + x
alice_x_share

23740629843736686616461

In [18]:
(bob_x_share + alice_x_share) % Q

5

So now, as you can see, both shares are wildly larger than the number being shared, meaning that individual shares no longer leak this inforation. However, all the properties we discussed earlier still hold! (addition, encryption, decryption, etc.)

# Project: Build Methods for Encrypt, Decrypt, and Add 

In this project, you must take the lessons we learned in the last section and write general methods for encrypt, decrypt, and add. Store shares for a variable in a tuple like so.

In [19]:
x_share = (2,5,7)

In [30]:
import random

In [31]:
Q = 23740629843760239486723

Even though normally those shares would be distributed amongst several workers, you can store them in ordered tuples like this for now :)

In [52]:
# try this project here!
def encrypt(x, n=3):
    #x -> number to encrypt, n -> number of shares
    
    shares = list()
    sum_shares = 0
    for i in range(n-1):
        shares.append(random.randint(0,Q))
        sum_shares += shares[i]
    shares.append(Q - sum_shares + x)
    
    #returns a tuple
    return tuple(shares)

In [32]:
def decrypt(shares):
    sum_shares = 0
    for share in shares:
        sum_shares += share
    
    #returns the decrypted value
    return sum_shares%Q

In [42]:
def add(a, b):
    c = []
    for i in range(len(a)):
        c.append((a[i] + b[i])%Q)
    
    #returns tuples after adding according to the protocol
    return tuple(c)

In [37]:
encrypt(5,3)

(22799488446845877398600, 10251400183293531386719, -9310258786379169298591)

In [38]:
decrypt(encrypt(5,3))

5

In [45]:
decrypt(add(encrypt(5,2), encrypt(10,2)))

15

# Lesson: Intro to Fixed Precision Encoding

As you may remember, our goal is to aggregate gradients using this new Secret Sharing technique. However, the protocol we've just explored in the last section uses positive integers. However, our neural network weights are NOT integers. Instead, our weights are decimals (floating point numbers).

Not a huge deal! We just need to use a fixed precision encoding, which lets us do computation over decimal numbers using integers!

In [46]:
BASE=10
PRECISION=4

In [47]:
def encode(x):
    return int(x * (BASE**PRECISION))%Q

In [48]:
def decode(x):
    return (x if x <= Q/2 else x - Q)/(BASE**PRECISION)

In [49]:
encode(3.5)

35000

In [50]:
decode(35000)

3.5

In [125]:
decode(23740629843760239486721)

-0.0002

In [53]:
x = encrypt(encode(5.5))
y = encrypt(encode(2.3))
z = add(x,y)
decode(decrypt(z))

7.8

# Lesson: Secret Sharing + Fixed Precision in PySyft

While writing things from scratch is certainly educational, PySyft makes a great deal of this much easier for us through its abstractions.

In [87]:
bob = bob.clear_objects()
alice = alice.clear_objects()
secure_worker = secure_worker.clear_objects()

In [88]:
x = torch.tensor([1,2,3,4,5])
x

tensor([1, 2, 3, 4, 5])

### Secret Sharing Using PySyft

We can share using the simple .share() method!

In [89]:
x = x.share(bob, alice, secure_worker)

In [92]:
x

(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:1641481574 -> bob:79012627860]
	-> (Wrapper)>[PointerTensor | me:50734409977 -> alice:63971355528]
	-> (Wrapper)>[PointerTensor | me:53423743829 -> secure_worker:86756564392]
	*crypto provider: me*

In [93]:
x.child

[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:1641481574 -> bob:79012627860]
	-> (Wrapper)>[PointerTensor | me:50734409977 -> alice:63971355528]
	-> (Wrapper)>[PointerTensor | me:53423743829 -> secure_worker:86756564392]
	*crypto provider: me*

In [94]:
x.child.child

{'bob': (Wrapper)>[PointerTensor | me:1641481574 -> bob:79012627860],
 'alice': (Wrapper)>[PointerTensor | me:50734409977 -> alice:63971355528],
 'secure_worker': (Wrapper)>[PointerTensor | me:53423743829 -> secure_worker:86756564392]}

In [103]:
x.child.child['bob'].child

[PointerTensor | me:1641481574 -> bob:79012627860]

In [104]:
bob._objects

{79012627860: tensor([  94949474364275961, 3673635822087122846, 3910706917311988501,
         2047214487750820284, 4022204515922830037])}

and as you can see, Bob now has one of the shares of x! Furthermore, we can still call addition in this state, and PySyft will automatically perform the remote execution for us!

In [105]:
x

(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:1641481574 -> bob:79012627860]
	-> (Wrapper)>[PointerTensor | me:50734409977 -> alice:63971355528]
	-> (Wrapper)>[PointerTensor | me:53423743829 -> secure_worker:86756564392]
	*crypto provider: me*

In [70]:
y = x + x

In [71]:
y

(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:30348120849 -> bob:2550516575]
	-> (Wrapper)>[PointerTensor | me:12290173874 -> alice:27797797579]
	-> (Wrapper)>[PointerTensor | me:21566393143 -> secure_worker:88418736395]
	*crypto provider: me*

In [72]:
y.get()

tensor([ 2,  4,  6,  8, 10])

### Fixed Precision using PySyft

We can also convert a tensor to fixed precision using .fix_precision()

In [74]:
x = torch.tensor([0.1,0.2,0.3])

In [75]:
x

tensor([0.1000, 0.2000, 0.3000])

In [76]:
x = x.fix_prec()

In [77]:
x

(Wrapper)>FixedPrecisionTensor>tensor([100, 200, 300])

In [78]:
x.child

FixedPrecisionTensor>tensor([100, 200, 300])

In [79]:
x.child.child

tensor([100, 200, 300])

In [80]:
y = x + x

In [81]:
y = y.float_prec()
y

tensor([0.2000, 0.4000, 0.6000])

### Shared Fixed Precision

And of course, we can combine the two!

In [119]:
x = torch.tensor([0.1, 0.2, 0.3])

In [120]:
x = x.fix_prec().share(bob, alice, secure_worker)

In [121]:
y = x + x

In [122]:
y.get().float_prec()

tensor([0.2000, 0.4000, 0.6000])

Make sure to make the point that people can see the model averages in the clear.

In [130]:
bob.clear_objects()
alice.clear_objects()
secure_worker.clear_objects()

<VirtualWorker id:secure_worker #objects:0>

# Final Project: Federated Learning with Encrypted Gradient Aggregation

In [127]:
data = torch.tensor([[1.,1], [0,1], [1,0], [0,0]], requires_grad=True)
target = torch.tensor([[1.], [1], [0], [0]], requires_grad=True)

In [134]:
bob.add_workers([alice])
alice.add_workers([bob])

<VirtualWorker id:alice #objects:0>

In [136]:
bobs_data = data[0:2].send(bob)
bobs_target = target[0:2].send(bob)

alices_data = data[2:].send(alice)
alices_target = target[2:].send(alice)

In [137]:
model = nn.Linear(2,1)

In [138]:
bobs_model = model.copy().send(bob)
alices_model = model.copy().send(alice)


bobs_opt = optim.SGD(params=bobs_model.parameters(), lr=0.1)
alices_opt = optim.SGD(params=alices_model.parameters(), lr=0.1)

for i in range(10):
    bobs_opt.zero_grad()
    bobs_pred = bobs_model(bobs_data)
    bobs_loss = ((bobs_pred - bobs_target)**2).sum()
    bobs_loss.backward()
    bobs_opt.step()
    
    bobs_loss = bobs_loss.get()
    
    alices_opt.zero_grad()
    alices_pred = alices_model(alices_data)
    alices_loss = ((alices_pred - alices_target)**2).sum()
    alices_loss.backward()
    alices_opt.step()
    
    alices_loss = alices_loss.get()

print(f"bobs_loss: {bobs_loss} alices_loss: {alices_loss}")



bobs_loss: 5.086875262350077e-06 alices_loss: 0.0011097200913354754


In [150]:
bobs_model.weight.data.fix_prec()

CannotRequestObjectAttribute: You called .get() on a pointer to a tensor attribute. This is not yet supported. Call .clone().get() instead.

In [151]:
a = bobs_model.weight.data.fix_prec().share(bob, alice, secure_worker)
alices_model.weight.data.fix_prec().share(bob, alice, secure_worker)

bobs_model.bias.data.fix_prec().share(bob, alice, secure_worker)
alices_model.bias.data.fix_prec().share(bob, alice, secure_worker)


AttributeError: 'numpy.ndarray' object has no attribute 'owner'